In [35]:
import pandas as pd
import numpy as np

df = pd.read_csv('npf_train.csv')


In [37]:
# Luodaan sarake class2 jossa on nonevent kun class4 == nonevent ja event kun class4 != nonevent

class2 = np.array(['event']*df.shape[0])
class2[df['class4']=='nonevent'] = 'nonevent'

df['class2'] = class2
df['class2'] = df['class2'].astype('category')

df.head()

,id,date,class4,partlybad,CO2168.mean,CO2168.std,CO2336.mean,CO2336.std,CO242.mean,CO242.std,...,T672.std,T84.mean,T84.std,UV_A.mean,UV_A.std,UV_B.mean,UV_B.std,CS.mean,CS.std,class2
0,1,2000-01-01,nonevent,False,384.462000,2.284996,384.164462,2.135062,385.274688,2.211695,...,0.525698,-12.422972,0.376324,1.635563,0.856948,0.026438,0.014617,0.003374,0.000733,nonev
1,2,2000-01-20,nonevent,False,374.884615,0.415185,374.703333,0.385179,375.621266,0.665720,...,0.373927,-8.351043,0.575679,1.441109,0.741088,0.022649,0.012479,0.001501,0.000572,nonev
2,3,2000-01-23,nonevent,False,373.496585,0.189497,373.382593,0.172958,373.961481,0.235107,...,0.965988,-9.651155,1.238891,2.677545,1.261612,0.044759,0.023748,0.000764,0.000048,nonev
3,4,2000-02-17,nonevent,False,378.600367,1.934180,378.464862,1.946536,379.785872,2.865022,...,0.122651,-0.829524,0.134191,2.261805,1.345651,0.030893,0.021903,0.002038,0.000751,nonev
4,5,2000-03-25,Ib,False,373.128684,1.096617,372.980000,1.047750,373.701830,1.259198,...,1.695622,-1.095864,2.090111,12.906779,7.022300,0.333523,0.239981,0.000662,0.000210,event


# Baseline malli
* Logistinen regressio
* https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
* ks. https://stackoverflow.com/questions/52670012/convergencewarning-liblinear-failed-to-converge-increase-the-number-of-iterati
* linkin takana vinkkejä 'failed to converge' varoitukseen
* https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-scaler

In [38]:
df.shape

(458, 105)

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop(['id', 'date', 'class4', 'partlybad', 'class2'], axis=1).to_numpy()
y = df['class2'].to_numpy()

scaler = StandardScaler().fit(X)

X_scaled = scaler.transform(X) # mean=0, std_deviation=1, Standard Normal Distribution

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
score = clf.score(X_test, y_test)
print(f"score: {score}")

clf.predict(X_test) #output järkevä

score: 0.8486842105263158


array(['event', 'nonev', 'event', 'event', 'event', 'nonev', 'nonev',
       'event', 'event', 'event', 'nonev', 'event', 'nonev', 'nonev',
       'nonev', 'event', 'nonev', 'nonev', 'nonev', 'nonev', 'nonev',
       'event', 'event', 'nonev', 'nonev', 'nonev', 'event', 'nonev',
       'event', 'event', 'nonev', 'event', 'event', 'event', 'event',
       'nonev', 'event', 'nonev', 'nonev', 'event', 'nonev', 'event',
       'event', 'nonev', 'nonev', 'nonev', 'event', 'nonev', 'nonev',
       'event', 'event', 'nonev', 'event', 'event', 'nonev', 'nonev',
       'event', 'event', 'nonev', 'nonev', 'event', 'nonev', 'event',
       'nonev', 'nonev', 'event', 'nonev', 'nonev', 'event', 'event',
       'event', 'nonev', 'nonev', 'nonev', 'nonev', 'event', 'event',
       'nonev', 'nonev', 'event', 'event', 'nonev', 'nonev', 'nonev',
       'nonev', 'nonev', 'nonev', 'event', 'nonev', 'event', 'nonev',
       'event', 'nonev', 'event', 'nonev', 'nonev', 'event', 'event',
       'event', 'eve

In [40]:
clf.predict_proba(X_test)

array([[9.63057486e-01, 3.69425142e-02],
       [3.84351102e-05, 9.99961565e-01],
       [9.25584426e-01, 7.44155740e-02],
       [8.01257093e-01, 1.98742907e-01],
       [9.99868162e-01, 1.31838496e-04],
       [2.08736322e-01, 7.91263678e-01],
       [1.43287688e-02, 9.85671231e-01],
       [9.75205504e-01, 2.47944959e-02],
       [9.66855302e-01, 3.31446977e-02],
       [9.78629136e-01, 2.13708642e-02],
       [2.43898557e-02, 9.75610144e-01],
       [9.98933421e-01, 1.06657872e-03],
       [3.41134344e-01, 6.58865656e-01],
       [1.78852703e-01, 8.21147297e-01],
       [1.34603495e-01, 8.65396505e-01],
       [9.74322997e-01, 2.56770034e-02],
       [1.47084216e-01, 8.52915784e-01],
       [4.64728411e-02, 9.53527159e-01],
       [1.58442734e-01, 8.41557266e-01],
       [2.69051834e-01, 7.30948166e-01],
       [7.55229712e-02, 9.24477029e-01],
       [9.13369002e-01, 8.66309978e-02],
       [5.26437268e-01, 4.73562732e-01],
       [4.56138647e-02, 9.54386135e-01],
       [4.927910

## Multilabel model
* Käyttää algoritmia 'newton-cg' -> tällä ei tullut convergence varoitusta

In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df['class4'] = df['class4'].astype('category')
X2 = df.drop(['id', 'date', 'class4', 'partlybad', 'class2'], axis=1).to_numpy()
y2 = df['class4'].to_numpy()

scaler2 = StandardScaler().fit(X2)

X2_scaled = scaler2.transform(X2) # mean=0, std_deviation=1, Standard Normal Distribution

X_train, X_test, y_train, y_test = train_test_split(X2_scaled, y2, test_size=0.33, random_state=42)

clf2 = LogisticRegression(random_state=0, solver='newton-cg', multi_class='multinomial').fit(X_train, y_train)
score = clf2.score(X_test, y_test)
print(f"score: {score}")

clf2.predict(X_test) #output järkevä

score: 0.618421052631579


array(['II', 'nonevent', 'II', 'Ib', 'II', 'Ia', 'nonevent', 'Ia', 'Ib',
       'Ib', 'nonevent', 'Ib', 'nonevent', 'nonevent', 'nonevent', 'II',
       'nonevent', 'nonevent', 'nonevent', 'nonevent', 'nonevent', 'Ib',
       'nonevent', 'nonevent', 'nonevent', 'nonevent', 'II', 'nonevent',
       'Ia', 'Ib', 'nonevent', 'Ib', 'Ib', 'Ib', 'II', 'nonevent', 'Ib',
       'nonevent', 'nonevent', 'Ib', 'nonevent', 'II', 'II', 'nonevent',
       'nonevent', 'nonevent', 'II', 'nonevent', 'nonevent', 'II', 'II',
       'nonevent', 'II', 'Ib', 'nonevent', 'nonevent', 'II', 'Ib',
       'nonevent', 'nonevent', 'II', 'nonevent', 'Ib', 'nonevent',
       'nonevent', 'II', 'nonevent', 'nonevent', 'II', 'II', 'Ib',
       'nonevent', 'nonevent', 'nonevent', 'nonevent', 'nonevent', 'II',
       'nonevent', 'nonevent', 'II', 'Ia', 'nonevent', 'nonevent',
       'nonevent', 'nonevent', 'nonevent', 'nonevent', 'Ib', 'nonevent',
       'Ib', 'nonevent', 'Ib', 'nonevent', 'II', 'nonevent', 'nonevent',
  

In [42]:
clf2.predict_proba(X_test)

array([[5.32233683e-01, 8.30286097e-04, 4.43727355e-01, 2.32086767e-02],
       [4.06214051e-05, 8.15389113e-07, 1.10806971e-06, 9.99957455e-01],
       [7.71623065e-01, 6.64944146e-03, 1.58840525e-01, 6.28869682e-02],
       [2.73706323e-01, 6.13972902e-03, 5.24627874e-01, 1.95526074e-01],
       [6.36205682e-01, 1.53381964e-02, 3.48352239e-01, 1.03883173e-04],
       [2.32861528e-03, 8.42388143e-01, 4.55300820e-03, 1.50730234e-01],
       [5.28234605e-03, 4.18476700e-03, 2.06717447e-04, 9.90326169e-01],
       [1.32361327e-01, 6.33397019e-01, 2.18520122e-01, 1.57215332e-02],
       [3.47727565e-01, 3.53406312e-02, 5.65064179e-01, 5.18676252e-02],
       [2.07704738e-02, 2.10258649e-04, 9.75343845e-01, 3.67542288e-03],
       [1.96055484e-02, 2.47285345e-04, 5.20013032e-03, 9.74947036e-01],
       [2.52130137e-01, 3.59298937e-02, 7.11152549e-01, 7.87421025e-04],
       [8.91975078e-02, 3.83455152e-01, 8.58358195e-03, 5.18763758e-01],
       [1.15982969e-01, 9.70694491e-04, 5.04885871e